In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "LDL_Cholesterol_Levels"
cohort = "GSE181339"

# Input paths
in_trait_dir = "../../input/GEO/LDL_Cholesterol_Levels"
in_cohort_dir = "../../input/GEO/LDL_Cholesterol_Levels/GSE181339"

# Output paths
out_data_file = "../../output/preprocess/LDL_Cholesterol_Levels/GSE181339.csv"
out_gene_data_file = "../../output/preprocess/LDL_Cholesterol_Levels/gene_data/GSE181339.csv"
out_clinical_data_file = "../../output/preprocess/LDL_Cholesterol_Levels/clinical_data/GSE181339.csv"
json_path = "../../output/preprocess/LDL_Cholesterol_Levels/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Study of the usefulness of human peripheral blood mononuclear cells for the analysis of metabolic recovery after weight loss (METAHEALTH-TEST)"
!Series_summary	"The aim of this study is to design and validate a test, METAHEALTH-TEST, based on gene expression analysis in blood cells, to quickly and easily analyse metabolic health. This test will be used to analyse metabolic improvement in overweight/obese individuals and in metabolically obese normal-weight (MONW) individuals after undergoing a weight loss intervention and/or an intervention for improvement in eating habits and lifestyle. Obesity and its medical complications are a serious health problem today. Using peripheral blood mononuclear cells (PBMC) as an easily obtainable source of transcriptomic biomarkers would allow to deepen into the knowledge of adaptations in response to increased adiposity that occur in internal homeostatic tissues, without the need of using invasive biopsies. More

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# Check gene expression data availability
is_gene_available = True  # Based on the background info, the dataset includes gene expression data from PBMC

# Identify the keys for trait, age, and gender data
trait_row = 1  # Group (OW/OB vs NW/MONW) is related to LDL cholesterol levels
age_row = 2    # Age is available in row 2
gender_row = 0  # Gender is available in row 0

# Convert trait function (binary: 1 for OW/OB, 0 for NW/MONW)
def convert_trait(value):
    if value is None:
        return None
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if 'OW/OB' in value:
        return 1  # Overweight/Obese is associated with higher LDL cholesterol
    elif 'NW' in value or 'MONW' in value:
        return 0  # Normal weight (includes metabolically obese normal weight)
    else:
        return None

# Convert age function (continuous)
def convert_age(value):
    if value is None:
        return None
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    try:
        return float(value)  # Convert to float for continuous variable
    except:
        return None

# Convert gender function (binary: 0 for female, 1 for male)
def convert_gender(value):
    if value is None:
        return None
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if value.lower() == 'man':
        return 1
    elif value.lower() == 'woman':
        return 0
    else:
        return None

# Check trait data availability
is_trait_available = trait_row is not None

# Save metadata for initial filtering
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# Since we don't have the actual clinical_data.csv file available, 
# we'll skip the part where we would extract and save clinical features.
# In a real scenario, we would load the file and process it.
print(f"Initial filtering for {cohort} completed.")
print(f"is_gene_available: {is_gene_available}")
print(f"is_trait_available: {is_trait_available}")


Initial filtering for GSE181339 completed.
is_gene_available: True
is_trait_available: True


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# Add diagnostic code to check file content and structure
print("Examining matrix file structure...")
with gzip.open(matrix_file, 'rt') as file:
    table_marker_found = False
    lines_read = 0
    for i, line in enumerate(file):
        lines_read += 1
        if '!series_matrix_table_begin' in line:
            table_marker_found = True
            print(f"Found table marker at line {i}")
            # Read a few lines after the marker to check data structure
            next_lines = [next(file, "").strip() for _ in range(5)]
            print("First few lines after marker:")
            for next_line in next_lines:
                print(next_line)
            break
        if i < 10:  # Print first few lines to see file structure
            print(f"Line {i}: {line.strip()}")
        if i > 100:  # Don't read the entire file
            break
    
    if not table_marker_found:
        print("Table marker '!series_matrix_table_begin' not found in first 100 lines")
    print(f"Total lines examined: {lines_read}")

# 2. Try extracting gene expression data from the matrix file again with better diagnostics
try:
    print("\nAttempting to extract gene data from matrix file...")
    gene_data = get_genetic_data(matrix_file)
    if gene_data.empty:
        print("Extracted gene expression data is empty")
        is_gene_available = False
    else:
        print(f"Successfully extracted gene data with {len(gene_data.index)} rows")
        print("First 20 gene IDs:")
        print(gene_data.index[:20])
        is_gene_available = True
except Exception as e:
    print(f"Error extracting gene data: {str(e)}")
    print("This dataset appears to have an empty or malformed gene expression matrix")
    is_gene_available = False

print(f"\nGene expression data available: {is_gene_available}")

# If data extraction failed, try an alternative approach using pandas directly
if not is_gene_available:
    print("\nTrying alternative approach to read gene expression data...")
    try:
        with gzip.open(matrix_file, 'rt') as file:
            # Skip lines until we find the marker
            for line in file:
                if '!series_matrix_table_begin' in line:
                    break
            
            # Try to read the data directly with pandas
            gene_data = pd.read_csv(file, sep='\t', index_col=0)
            
            if not gene_data.empty:
                print(f"Successfully extracted gene data with alternative method: {gene_data.shape}")
                print("First 20 gene IDs:")
                print(gene_data.index[:20])
                is_gene_available = True
            else:
                print("Alternative extraction method also produced empty data")
    except Exception as e:
        print(f"Alternative extraction failed: {str(e)}")


Examining matrix file structure...
Line 0: !Series_title	"Study of the usefulness of human peripheral blood mononuclear cells for the analysis of metabolic recovery after weight loss (METAHEALTH-TEST)"
Line 1: !Series_geo_accession	"GSE181339"
Line 2: !Series_status	"Public on Oct 11 2023"
Line 3: !Series_submission_date	"Aug 02 2021"
Line 4: !Series_last_update_date	"Oct 12 2023"
Line 5: !Series_pubmed_id	"36564895"
Line 6: !Series_summary	"The aim of this study is to design and validate a test, METAHEALTH-TEST, based on gene expression analysis in blood cells, to quickly and easily analyse metabolic health. This test will be used to analyse metabolic improvement in overweight/obese individuals and in metabolically obese normal-weight (MONW) individuals after undergoing a weight loss intervention and/or an intervention for improvement in eating habits and lifestyle. Obesity and its medical complications are a serious health problem today. Using peripheral blood mononuclear cells (PBMC

### Step 4: Gene Identifier Review

In [5]:
# Examining the gene identifiers
# The identifiers in this dataset are numeric values like '7', '8', '15', etc.
# These are not standard human gene symbols (which would typically be like BRCA1, TP53, etc.)
# These appear to be probe IDs from a microarray platform that need mapping to gene symbols

# For Agilent microarrays (likely used here based on the file names in the metadata), 
# these would be probe IDs that require mapping to gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Extract gene annotation data from the SOFT file
print("Extracting gene annotation data from SOFT file...")
try:
    # Use the library function to extract gene annotation
    gene_annotation = get_gene_annotation(soft_file)
    print(f"Successfully extracted gene annotation data with {len(gene_annotation.index)} rows")
    
    # Preview the annotation DataFrame
    print("\nGene annotation preview (first few rows):")
    print(preview_df(gene_annotation))
    
    # Show column names to help identify which columns we need for mapping
    print("\nColumn names in gene annotation data:")
    print(gene_annotation.columns.tolist())
    
    # Check for relevant mapping columns
    if 'GB_ACC' in gene_annotation.columns:
        print("\nThe dataset contains GenBank accessions (GB_ACC) that could be used for gene mapping.")
        # Count non-null values in GB_ACC column
        non_null_count = gene_annotation['GB_ACC'].count()
        print(f"Number of rows with GenBank accessions: {non_null_count} out of {len(gene_annotation)}")
    
    if 'SPOT_ID' in gene_annotation.columns:
        print("\nThe dataset contains genomic regions (SPOT_ID) that could be used for location-based gene mapping.")
        print("Example SPOT_ID format:", gene_annotation['SPOT_ID'].iloc[0])
    
except Exception as e:
    print(f"Error processing gene annotation data: {e}")
    is_gene_available = False


Extracting gene annotation data from SOFT file...


Successfully extracted gene annotation data with 1815168 rows

Gene annotation preview (first few rows):
{'ID': ['1', '2', '3', '4', '5'], 'COL': ['192', '192', '192', '192', '192'], 'ROW': [328.0, 326.0, 324.0, 322.0, 320.0], 'NAME': ['GE_BrightCorner', 'DarkCorner', 'DarkCorner', 'A_21_P0014386', 'A_33_P3396872'], 'SPOT_ID': ['GE_BrightCorner', 'DarkCorner', 'DarkCorner', 'A_21_P0014386', 'A_33_P3396872'], 'CONTROL_TYPE': ['pos', 'pos', 'pos', 'FALSE', 'FALSE'], 'REFSEQ': [nan, nan, nan, nan, 'NM_001105533'], 'GB_ACC': [nan, nan, nan, nan, 'NM_001105533'], 'LOCUSLINK_ID': [nan, nan, nan, nan, 79974.0], 'GENE_SYMBOL': [nan, nan, nan, nan, 'CPED1'], 'GENE_NAME': [nan, nan, nan, nan, 'cadherin-like and PC-esterase domain containing 1'], 'UNIGENE_ID': [nan, nan, nan, nan, 'Hs.189652'], 'ENSEMBL_ID': [nan, nan, nan, nan, nan], 'TIGR_ID': [nan, nan, nan, nan, nan], 'ACCESSION_STRING': [nan, nan, nan, nan, 'ref|NM_001105533|gb|AK025639|gb|BC030538|tc|THC2601673'], 'CHROMOSOMAL_LOCATION': [n

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine which columns to use for gene ID mapping
# Based on gene_annotation preview, 'ID' column contains numeric identifiers matching gene expression IDs
# 'GENE_SYMBOL' column contains the target gene symbols

# Extract gene mapping from annotation using these columns
print("Creating gene mapping from ID to GENE_SYMBOL...")
mapping_df = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='GENE_SYMBOL')
print(f"Created mapping with {len(mapping_df)} entries")

# Preview first few rows of the mapping
print("\nPreview of gene mapping (first few rows):")
print(preview_df(mapping_df.head()))

# 2. Apply the gene mapping to convert probe-level measurements to gene expression
print("\nConverting probe-level measurements to gene expression data...")
gene_data = apply_gene_mapping(expression_df=gene_data, mapping_df=mapping_df)
print(f"Generated gene expression data for {len(gene_data.index)} genes across {len(gene_data.columns)} samples")

# 3. Preview the gene expression data
print("\nPreview of gene expression data (first few genes):")
print(preview_df(gene_data.head()))

# 4. Check if the mapping was successful by looking at the presence of common gene symbols
common_genes = ['TP53', 'BRCA1', 'TNF', 'IL6', 'APOE']
found_genes = [gene for gene in common_genes if gene in gene_data.index]
print(f"\nFound {len(found_genes)} out of {len(common_genes)} common genes in the mapped data")
if found_genes:
    print(f"Found genes: {found_genes}")

# Normalize gene symbols in index to ensure consistency
print("\nNormalizing gene symbols...")
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"After normalization: {len(gene_data.index)} unique genes")


Creating gene mapping from ID to GENE_SYMBOL...
Created mapping with 51544 entries

Preview of gene mapping (first few rows):
{'ID': ['5', '6', '7', '8', '12'], 'Gene': ['CPED1', 'BCOR', 'CHAC2', 'IFI30', 'GPR146']}

Converting probe-level measurements to gene expression data...
Generated gene expression data for 13263 genes across 78 samples

Preview of gene expression data (first few genes):
{'GSM5494930': [9.356389, 6.588705, 20.173181, 6.087023, 8.855058], 'GSM5494931': [9.580217, 6.861172, 17.179827, 5.95844, 8.172307], 'GSM5494932': [9.920784, 7.055549, 18.935323, 6.690681, 8.768802], 'GSM5494933': [9.504974, 6.792186, 15.861170000000001, 5.814862, 8.708854], 'GSM5494934': [9.533504, 7.192053, 19.192128, 5.822462, 8.534389], 'GSM5494935': [9.926714, 7.000017, 18.942311, 5.521768, 8.529483], 'GSM5494936': [10.22561, 7.219546, 17.853802, 5.832344, 8.113828], 'GSM5494937': [9.708488, 6.974349, 19.511087, 5.259415, 8.449762], 'GSM5494938': [9.759847, 7.343875, 16.303942, 6.574513, 8.

After normalization: 12283 unique genes


### Step 7: Data Normalization and Linking

In [8]:
# 1. Save the normalized gene expression data from the previous step
print("\nSaving normalized gene data...")
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Extract clinical data from the matrix file
print("\nExtracting clinical data...")
try:
    # Get the file paths again to make sure we have them
    soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
    
    # Extract background information and clinical data
    background_info, clinical_data = get_background_and_clinical_data(
        matrix_file, 
        prefixes_a=['!Series_title', '!Series_summary', '!Series_overall_design'],
        prefixes_b=['!Sample_geo_accession', '!Sample_characteristics_ch1']
    )
    
    # Process clinical data using trait information from Step 2
    trait_row = 1  # Based on analysis in step 2 - group (OW/OB vs NW/MONW)
    gender_row = 0  # Gender data
    age_row = 2     # Age data
    
    # Define conversion functions based on Step 2
    def convert_trait(value):
        """Convert trait value (binary: 1 for OW/OB, 0 for NW/MONW)"""
        if pd.isna(value):
            return None
        
        # Extract value after colon if present
        if ':' in value:
            value = value.split(':', 1)[1].strip()
        
        if 'OW/OB' in value:
            return 1  # Overweight/Obese is associated with higher LDL cholesterol
        elif 'NW' in value or 'MONW' in value:
            return 0  # Normal weight (includes metabolically obese normal weight)
        else:
            return None

    def convert_gender(value):
        """Convert gender value to binary (0: female, 1: male)"""
        if pd.isna(value):
            return None
        
        # Extract value after colon if present
        if ':' in value:
            value = value.split(':', 1)[1].strip()
        
        # Convert gender
        if value.lower() == 'woman':
            return 0
        elif value.lower() == 'man':
            return 1
        else:
            return None
    
    def convert_age(value):
        """Convert age value to float"""
        if pd.isna(value):
            return None
        
        # Extract value after colon if present
        if ':' in value:
            value = value.split(':', 1)[1].strip()
        
        try:
            return float(value)  # Convert to float for continuous variable
        except:
            return None
    
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Save the clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")
    print(f"Clinical data shape: {selected_clinical_df.shape}")
    
    # Check if we have valid trait information
    is_trait_available = trait_row is not None and not selected_clinical_df.loc[trait].isnull().all()
    print(f"Trait information available: {is_trait_available}")
    
except Exception as e:
    print(f"Error extracting clinical data: {e}")
    is_trait_available = False
    selected_clinical_df = pd.DataFrame()

# 3. Link clinical and genetic data
print("\nLinking clinical and genetic data...")
try:
    if is_trait_available and not selected_clinical_df.empty:
        # Link clinical and genetic data
        linked_data = geo_link_clinical_genetic_data(selected_clinical_df, gene_data)
        print(f"Created linked data with {linked_data.shape[0]} samples and {linked_data.shape[1]} features")
    else:
        print("Cannot link data: clinical data is not available")
        linked_data = pd.DataFrame()
        is_trait_available = False
except Exception as e:
    print(f"Error linking clinical and genetic data: {e}")
    is_trait_available = False
    linked_data = pd.DataFrame()

# 4. Handle missing values in the linked data
if is_trait_available and not linked_data.empty:
    print("\nHandling missing values...")
    try:
        # Rename the first column to the trait name for consistency
        if linked_data.columns[0] != trait:
            linked_data = linked_data.rename(columns={linked_data.columns[0]: trait})
        
        linked_data = handle_missing_values(linked_data, trait)
        print(f"After handling missing values: {linked_data.shape[0]} samples and {linked_data.shape[1]} features")
    except Exception as e:
        print(f"Error handling missing values: {e}")
        
    # 5. Determine whether the trait and demographic features are biased
    print("\nEvaluating feature bias...")
    try:
        is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
        print(f"Trait bias determination: {is_biased}")
        print(f"Final linked data shape: {linked_data.shape[0]} samples and {linked_data.shape[1]} features")
    except Exception as e:
        print(f"Error evaluating feature bias: {e}")
        is_biased = True
else:
    print("\nSkipping missing value handling and bias evaluation as linked data is not available")
    is_biased = True

# 6. Validate and save cohort information
print("\nPerforming final validation...")
note = ""
if not is_trait_available:
    note = "Dataset does not contain required trait information"
elif is_biased:
    note = "Dataset has severe bias in the trait distribution"

is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available,
    is_biased=is_biased,
    df=linked_data,
    note=note
)

# 7. Save the linked data if usable
print(f"\nDataset usability for {trait} association studies: {is_usable}")
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Final linked data saved to {out_data_file}")
else:
    if note:
        print(f"Reason: {note}")
    else:
        print("Dataset does not meet quality criteria for the specified trait")


Saving normalized gene data...


Normalized gene data saved to ../../output/preprocess/LDL_Cholesterol_Levels/gene_data/GSE181339.csv

Extracting clinical data...
Clinical data saved to ../../output/preprocess/LDL_Cholesterol_Levels/clinical_data/GSE181339.csv
Clinical data shape: (3, 78)
Trait information available: True

Linking clinical and genetic data...
Created linked data with 78 samples and 12286 features

Handling missing values...


After handling missing values: 78 samples and 12286 features

Evaluating feature bias...
For the feature 'LDL_Cholesterol_Levels', the least common label is '0.0' with 30 occurrences. This represents 38.46% of the dataset.
The distribution of the feature 'LDL_Cholesterol_Levels' in this dataset is fine.

Quartiles for 'Age':
  25%: 8.0
  50% (Median): 11.0
  75%: 18.75
Min: 1.0
Max: 30.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '1.0' with 39 occurrences. This represents 50.00% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

Trait bias determination: False
Final linked data shape: 78 samples and 12286 features

Performing final validation...

Dataset usability for LDL_Cholesterol_Levels association studies: True


Final linked data saved to ../../output/preprocess/LDL_Cholesterol_Levels/GSE181339.csv
